# Exploring patient matching with unsupervised learning : Expectation-Maximisation and K-means
This notebook introduces key concepts of patient matching while demonstrating those concepts using python

***********************************************

This notebook follows  step by step the record linkage process , providing minimum exaplanation and assuming  knowledge of the process. For more detailed information, please consult the reference section


## Record linkage process (deduplication)

In [135]:
from IPython.display import display
import warnings
import numpy as np
import pandas as pd
import recordlinkage 
from recordlinkage.index import Block
from recordlinkage.preprocessing import phonetic
warnings.filterwarnings('ignore')

In [136]:
# file to deduplicate
IMPORT_FILE_TO_DEDUPLICATE = './data/dataset_febrl3.csv'


In this notebook we use  synthetic dataset from The Python Record Linkage Toolkit (PRLT). The PRLT contains several open public synthetic datasets. The package is distributed with a  four synthetic datasets. For this project we will use The Freely Extensible Biomedical Record Linkage (Febrl) dataset 3 . Dataset 3 (FEBRL3) contains 5000 records (2000 originals and 3000 duplicates).

For more info : [Synthetic datasets](./https://recordlinkage.readthedocs.io/en/latest/ref-datasets.html "Title")

In [137]:
## add columns
df_a = pd.read_csv(IMPORT_FILE_TO_DEDUPLICATE)
df_a = df_a.set_index('rec_id')
print('Number of records :', len(df_a))
df_a.head()


Number of records : 5000


,given_name,surname,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,
rec-1496-org,mitchell,green,wallaby place,delmar,cleveland,2119,sa,19560409.0,1804974
rec-552-dup-3,harley,mccarthy,pridhamstreet,milton,marsden,3165,nsw,19080419.0,6089216
rec-988-dup-1,madeline,mason,hoseason street,lakefront retrmnt vlge,granville,4881,nsw,19081128.0,2185997
rec-1716-dup-1,isabelle,NaN,gundulu place,currin ga,utakarra,2193,wa,19921119.0,4314184
rec-1213-org,taylor,hathaway,yuranigh court,brentwood vlge,NaN,4220,nsw,19991207.0,9144092


## Pre-processing and standardization

This is the first step of the Record linkage process. The main task of **data cleaning and standardization** is the conversion of the raw input data into well defined, consistent forms, as well as the resolution of inconsistencies in the way information is represented and encoded. (source:)

We have split the date of birth in 3 columns for more easy comparison, also we have calculated the metaphone of the given name and surname respectively.

**Metaphone** is a phonetic encoding algorithm used to encode the way words an syllable are pronounces to help **reduce minor typographical error**.The output of a phonetic algorithm is an intentionally approximate phonetic representation of the word. With application still limited to English words Metaphone is an improvement on the Soundex algorihtm .


In [138]:
# convert date of birth as string
df_a['date_of_birth'] = pd.to_datetime(df_a['date_of_birth'],format='%Y%m%d', errors='coerce')
df_a['YearB'] = df_a['date_of_birth'].dt.year.astype('Int64') 
df_a['MonthB'] = df_a['date_of_birth'].dt.month.astype('Int64') 
df_a['DayB'] = df_a['date_of_birth'].dt.day.astype('Int64') 

df_a['metaphone_given_name'] = phonetic(df_a['given_name'], method='metaphone')
df_a['metaphone_surname'] = phonetic(df_a['surname'], method='metaphone')
#df_a.sort_values(['given_name'])

## Blocking and Indexing

The second step of the process called ***blocking or indexing** try to reduce the number of records we need to compare. The idea is instead of comparing all records of the dataset between themselves we want to compare only the records that are most likely to be matched. 

As example you can decide to compare only patiend with the same : first name, last name and date of birth. This combination of fields is called a **  blocking key**. Using  a blocking key provide a reduce set of record pairs. In this notebook we use multiple blocking keys and consider the  ** union  ** of all the results set of candidate record pairs to evaluate for matching in the next steps.

Please note the use of the **metaphone** algorithm here instead of the exact value.This takes into account typrographic errors in the names and provide a wider range of candidiate record pairs.

In [139]:
indexer = recordlinkage.Index()
        
# soundex firstname, methapone surname, exact date of birth
indexer.add(Block(['metaphone_given_name','metaphone_surname','date_of_birth']))
# soundex firstname , day of birth
indexer.add(Block(['metaphone_given_name','DayB']))
#soundex firstname , month of birth
indexer.add(Block(['metaphone_given_name','MonthB']))
# metaphone surname, year of birth 
indexer.add(Block(['metaphone_surname','YearB']))
# ssn
indexer.add(Block(['soc_sec_id']))

candidate_record_pairs = indexer.index(df_a)

print("Number of record pairs :",len(candidate_record_pairs))
candidate_record_pairs.to_frame(index=False).head()

Number of record pairs : 12833


,rec_id_1,rec_id_2
0,rec-0-org,rec-1023-org
1,rec-0-org,rec-1540-dup-1
2,rec-1-org,rec-1643-org
3,rec-1-org,rec-1986-org
4,rec-1-org,rec-41-org


In PRLT Phonetic encoding possible options are “soundex”, “nysiis”, “metaphone” or “match_rating”.**Other phonetic algorithm not included in PRLT : double-metaphne, phonix , phonex, OCNA, Fuzzy soundex (Christen 2012)**

## Comparison

Identifying the similarity between records pairs to create a comparison vectors. 

The previous step provided us a list of record pairs. In this step we compare the corresponding fields of each record pair using string distance algorithm.
Jarowinkler and Levenshtein generate a **score between 0 and 1** that is binarized based on the threshold. In our case if the **score >0.85** we say there's agreeement (1) if not there's disagreement (0).

The output of the comparison is the **comparison vector** that will be used for classification.

In [140]:

compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name', method='jarowinkler', threshold = 0.85, label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',threshold = 0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('soc_sec_id', 'soc_sec_id', label='soc_sec_id')
compare_cl.string('address_1', 'address_1', method ='levenshtein' ,threshold = 0.85, label='address_1')
compare_cl.string('address_2', 'address_2', method ='levenshtein' ,threshold = 0.85, label='address_2')
compare_cl.string('suburb', 'suburb', method ='levenshtein' ,threshold = 0.85, label='suburb')
compare_cl.exact('postcode', 'postcode', label='postcode')
compare_cl.exact('state', 'state', label='state')

features = compare_cl.compute(candidate_record_pairs, df_a)
features.head()


given_name  surname  date_of_birth  soc_sec_id  \
rec_id_1  rec_id_2                                                         
rec-0-org rec-1023-org           0.0      0.0              0           0   
          rec-1540-dup-1         0.0      0.0              0           0   
rec-1-org rec-1643-org           0.0      0.0              0           0   
          rec-1986-org           1.0      0.0              0           0   
          rec-41-org             0.0      0.0              0           0   

                          address_1  address_2  suburb  postcode  state  
rec_id_1  rec_id_2                                                       
rec-0-org rec-1023-org          0.0        0.0     0.0         0      0  
          rec-1540-dup-1        0.0        0.0     0.0         0      1  
rec-1-org rec-1643-org          0.0        0.0     0.0         0      0  
          rec-1986-org          0.0        0.0     0.0         0      0  
          rec-41-org            0.0        0.0     0.0         0      0

## Classification

Based on comparison results, this step uses a classification algorithm to classify candidate records pairs in: matches, non-matches or potential matches.

Probabilistic matching is based on a probability model that designates record pairs as matches, possible matches, or non-matches based on calculation of linkage scores and application of decision rules about these scores to define true matches. 


### ECM Classifier

** EM-Algorithm ** :
This Expectation-Maximisation (EM) algorithm is an unsupervised probabilistic algorithm which **automatically estimate a threshold for the likelihood score to decide a match and non-match**. This do not need training data.

References :
* Herzog, Thomas N, Fritz J Scheuren and William E Winkler. 2007. Data quality and record linkage techniques. Vol. 1 Springer.
* Fellegi, Ivan P and Alan B Sunter. 1969. “A theory for record linkage.” Journal of the American Statistical Association 64(328):1183–1210.

In [141]:
ecm = recordlinkage.ECMClassifier()
matches = ecm.fit_predict(features)
print("Number of matched record pairs :",len(matches))
matches.to_frame(index=False).head()

Number of matched record pairs : 6276


,rec_id_1,rec_id_2
0,rec-10-dup-0,rec-10-dup-2
1,rec-10-dup-1,rec-10-dup-0
2,rec-10-dup-1,rec-10-dup-2
3,rec-10-org,rec-10-dup-0
4,rec-10-org,rec-10-dup-1


### K-means classifier

In [142]:
kmeans = recordlinkage.KMeansClassifier()
matches_kmeans = kmeans.fit_predict(features)

# The predicted number of matches
type(matches_kmeans)
print("Number of matched record pairs :",len(matches_kmeans))
matches_kmeans.to_frame(index=False).head()

Number of matched record pairs : 6228


,rec_id_1,rec_id_2
0,rec-10-dup-0,rec-10-dup-2
1,rec-10-dup-1,rec-10-dup-0
2,rec-10-dup-1,rec-10-dup-2
3,rec-10-org,rec-10-dup-0
4,rec-10-org,rec-10-dup-1


## Evaluation

Comparing match results with the known ground truth or gold standard to mesaure the performance of the matching process.


### Gold standard 
The main objective of evaluation techniques is to achieve **high matching quality** in order to assess  the quality of the matched  data for a certain project ground-truth data also known as gold standard is required.

There are several approches of how ground-thruth data can be generated. In this notebook the gold standard data was generated as part of the synthetic data used for matching.

In [143]:
# gold_ standard or known truth
IMPORT_FILE_GOLD_STANDARD = './data/dataset_febrl3_true_links.csv'

In [144]:
df_true_links = pd.read_csv(IMPORT_FILE_GOLD_STANDARD)
df_true_links.columns=['rec_id_1','rec_id_2']
df_true_links.set_index(['rec_id_1','rec_id_2'],inplace=True)
df_true_links.head()

Empty DataFrame
Columns: []
Index: [(rec-552-dup-1, rec-552-dup-3), (rec-552-dup-0, rec-552-dup-3), (rec-552-dup-0, rec-552-dup-1), (rec-552-org, rec-552-dup-3), (rec-552-org, rec-552-dup-1)]

In [145]:
def metrics(links_true,links_pred,pairs):
    if len(links_pred) > 0 :
        matrix  = recordlinkage.confusion_matrix(links_true, links_pred, len(pairs))
            
        # precision
        precision  = recordlinkage.precision(links_true, links_pred)

         #precision
        recall  = recordlinkage.recall(links_true, links_pred)

        # The F-score for this classification is
        fscore = recordlinkage.fscore(links_true,links_pred)
        
        return {'precision':precision, 'recall':recall,'fscore':fscore}
    else :
        return {'precision':0, 'recall':0,'fscore':0}

In [151]:
## Create Function to Print Results
def get_results(metrics):
    print("\n{0:20}    {1:6}    {2:6}    {3:6}".format('Matching ','Precision','Recall','Fscore'))
    print('------------------------------------------------------')
    for i in metrics.keys():
        print("{0:20}    {1:<6.4}      {2:<6.4}      {3:<6.4}".format(i,metrics[i]['precision'],
                                                                      metrics[i]['recall'],
                                                                      float(metrics[i]['fscore'])))

In [152]:
results_score = {}

results_score['ECM'] =  metrics(df_true_links,matches,features)
results_score['K-means'] = metrics(df_true_links,matches_kmeans,features)

In [153]:
get_results(results_score)


Matching                Precision    Recall    Fscore
------------------------------------------------------
ECM                     1.0         0.9599      0.9796
K-means                 1.0         0.9526      0.9757


## References

* de Bruin, Jonathan. 2017. “Record Linkage. Python library. Version 0.8.1.” https://recordlinkage.readthedocs.io/.

* Herzog, Thomas N, Fritz J Scheuren and William E Winkler. 2007. Data quality and record linkage techniques. Vol. 1 Springer.

* Fellegi, Ivan P and Alan B Sunter. 1969. “A theory for record linkage.” Journal of the American Statistical Association 64(328):1183–1210.

* Christen, Peter. 2012. Data matching: concepts and techniques for record linkage, entity resolution, and duplicate detection. Springer Science & Business Media.